In [10]:
import pandas as pd
import os
import numpy as np

In [7]:
# Change working directory
os.chdir('X:\\Academico\\CUNEF\\TFM\\predictive_maintenance\\src')

# Listing all the data raw documents
data_files = os.listdir('data\\raw')